In [492]:
import re
import numpy as np
import json

In [9]:
with open("linker-type-dependencies.txt", "r") as f:
    lines = f.readlines()

In [223]:
def split_dep(dep_ls):
    dep_dict = {}
    for dep in dep_ls:
        dep = dep.split(":")
        name = dep[0]
        dep = ":".join(dep[1:])
        if name not in dep_dict:
            dep_dict[name] = [dep]
        else:
            dep_dict[name].append(dep)
    return dep_dict

In [507]:
def method_name(name):
    d = name.split(" ")
    return " ".join(d[:-2])

def assembly_name(name):
    d = name.split(",")[0]
    return d

def enum_dep(dep_dict):
    all_dep = []
    for key in dep_dict:
        dep = dep_dict[key]
        if key in ["Method", "TypeDef"]: 
            all_dep += [method_name(d) for d in dep]
        elif key == "Assembly":
            all_dep += [d.split(",")[0] for d in dep]
#         else:
#             all_dep += [d.split(" ")[0] for d in dep]
    return all_dep


In [508]:
i = 0
dep_mod = {}
length = len(lines)
types = set()
while i < length:
    line = lines[i]
    if "--- TypeDef:" in line:
        name = line[12:].split(' ')[0]
        types.add(name)
        dep = []
        
        i += 1
        while i < length and "--- TypeDef:" not in lines[i] :
            if "deps]" in lines[i]:
                line = lines[i].replace("\t| ", "").replace("\n", "")
                dep.append(line)
                
                field = line.split(":")[0]
                if field in ["Method", "TypeDef"]:
                    assembly = " ".join(line[len(field) + 1:].split(" ")[:-2])
                elif field == "Assembly":
                    assembly = line.split(":")[1].split(",")[0]
                types.add(assembly)
            i += 1
        dep_mod[name] = split_dep(dep)
    else:
        i += 1

In [564]:
ex = dep_mod['System.Runtime.InteropServices.ComVisibleAttribute']
print(enum_dep(ex))

['System.Object', 'System.ValueType', 'System.IntPtr', 'System.Runtime.Remoting.Proxies.RealProxy', 'System.Runtime.Remoting.Contexts.Context', 'System.UIntPtr', 'System.Runtime.Remoting.Messaging.IMessageSink', 'System.Runtime.Remoting.Contexts.Context', 'System.Runtime.Remoting.Contexts.IContextProperty', 'System.Runtime.Remoting.Contexts.Context', 'System.MarshalByRefObject', 'System.ContextBoundObject', 'System.Runtime.Remoting.Contexts.ContextCallbackObject:mscorlib.dll', 'System.Runtime.Remoting.Contexts.Context', 'System.Runtime.Remoting.IRemotingTypeInfo', 'System.Exception', 'System.Diagnostics.StackTrace', 'System.Exception', 'System.TypeLoadException', 'System.Runtime.InteropServices._Attribute', 'System.Runtime.InteropServices.ComDefaultInterfaceAttribute', 'System.Attribute', 'System.Attribute', 'System.Enum', 'System.TypedReference', 'System.RuntimeTypeHandle', 'System.AppDomain', 'System._AppDomain', 'System.Runtime.InteropServices.ComDefaultInterfaceAttribute', 'System.

In [510]:
print(ex)

{'TypeDef': ['System.Delegate [4 deps]'], 'Assembly': ['mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089 [2 deps]']}


In [511]:
types = list(types)
num_types = len(types)

In [513]:
num_types

6456

In [557]:
dep_mat = np.zeros((num_types, num_types))
for name in dep_mod:
    i = types.index(name)
    dep = enum_dep(dep_mod[name])
    for d in dep:
        dep_mat[i][types.index(d)] = 1

In [567]:
a = np.sum(dep_mat, axis=1)
max_args = a.argsort()[-75:][::-1]
max_args

array([1295, 3988, 5076, 6070, 2581, 4466, 2951, 6000, 4847, 5874, 3235,
        571, 2579, 4144, 5786,  642, 1268, 3220, 2917, 5434, 5936, 3245,
        874,  311, 3701,  543,  208,  467, 4663,  330, 2752, 1052, 4310,
       5174, 3782, 3677, 3558, 5942, 1798, 5551, 4453, 4555,  677, 2209,
       4645, 5160, 3158, 5428, 4246, 3143, 1713,  345, 4642, 2948, 1216,
       2870,  382, 4785, 1038,  725, 1404, 5457, 5723, 6417,  209, 3112,
       3941, 1614, 5591, 2217, 2788,  521, 4240, 3204, 5734])

In [566]:
sum(dep_mat[1295])
#types[436]

916.0

In [565]:
types.index("System.Runtime.InteropServices.ComVisibleAttribute")

1295

In [570]:
con_types = np.take(types, max_args)
con_types

array(['System.Runtime.InteropServices.ComVisibleAttribute',
       'System.CLSCompliantAttribute',
       'System.Runtime.CompilerServices.CompilerGeneratedAttribute',
       'System.AttributeUsageAttribute',
       'System.Runtime.ConstrainedExecution.ReliabilityContractAttribute',
       'System.FlagsAttribute',
       'System.Runtime.CompilerServices.ExtensionAttribute',
       'System.Reflection.DefaultMemberAttribute',
       'System.ObsoleteAttribute',
       'System.Runtime.Versioning.NonVersionableAttribute:mscorlib.dll',
       'System.Diagnostics.DebuggerDisplayAttribute',
       'System.MonoTODOAttribute:mscorlib.dll',
       'System.Diagnostics.DebuggerHiddenAttribute',
       'System.ParamArrayAttribute',
       'System.Runtime.CompilerServices.IsReadOnlyAttribute',
       'System.Diagnostics.DebuggerTypeProxyAttribute',
       'System.Diagnostics.DebuggerStepThroughAttribute',
       'System.Runtime.CompilerServices.IsByRefLikeAttribute',
       'System.Runtime.Versionin

In [574]:
con_dep_mat = np.take(dep_mat, max_args, axis=0)
con_dep_mat = np.take(con_dep_mat, max_args, axis=1)
#con_dep_mat = con_dep_mat[:, :75]
con_dep_mat.shape

(75, 75)

In [575]:
data = {
    "matrix": con_dep_mat.astype(int).tolist(),
    "packageNames": con_types.tolist()
    
}

with open("../web/json/data.json", "w") as write_file:
    json.dump(data, write_file)
